## Fase 2

- Preparação e pre-processamento dos dados para a aplicação nos modelos.
- Geração de ambiente de testes básico para os modelos
- Filtragem de modelos viáveis

In [1]:
from src.preprocessing import *

#### Passo 1

Substituir colunas "open" e "close" pela media e diferenca delas.

Uma pergunta que deve ser feita é. Como lidar com o fato de existirem tantas empresas. Será que os dados delas devem ser todos agregados em um só conjunto para a Bolsa? Ou será que é possível tratá-los do jeito que estão?

Um jeito melhor de formular essa pergunta é o seguinte. Será que o símbolo da empresas (atributo categórico) deve ser um dos parâmetros do modelo?

Para avaliar uma resposta, podemos colocar alguns fatores em jogo.

- Complexidade: qual seria mais difícil de construir, o agregado ou o sem agregação?
- Utilidade (em vista do objetivo de negócio): Note que o objetivo de negócio menciona, e com razão, "robustez". É bem claro que um preditor que leve em conta especificamente a qual empresa se refere, terá melhores resultados que um preditor que agrega tudo (perdendo tantos graus de liberdade). Além disso, como demonstrado na fase 1 pela comparação entre as empresas IBM e Fox, cada uma tem suas peculiaridades. Analisá-las em conjunto pode trazer um grande prejuízo para a acurácia do modelo.

Essa discussão levanta um ponto interessante. O banco de dados original consiste não somente no arquivo "prices.csv" que estamos utilizando, mas também de outros arquivos, por exemplo tabelas que trazem informações cruciais sobre a empresa. Esse ponto é interessante porque, das desvantagens de se tomar o símbolo da empresa como parâmetro para o modelo, se destaca a incapacidade de generalizar para novas empresas que surgirem, ou para mudanças de nome (mudanças do símbolo da empresa teriam que ser tratadas manualmente). Seria interessante se as empresas pudessem ser classificadas por setor (e.g. IT, media, etc), por exemplo, como um passo de pre-processamento de entradas para o modelo.

OK. Acho que o que precisamos fazer é codificar essas variáveis categóricas com o que se pode encontrar no arquivo "fundamentals.csv".

**One question that I would like to raise here is,
is our model really going to need this transformation?**

We have decided to leave the consideration of low & high to after the model testing framework is built.

#### Passo 2

> Compilar uma lista de modelos compatı́veis com as observações feitas até o
momento. Comparar analiticamente estes modelos, deixando na lista somente modelos
que se confundirem em termos de qualidade esperada. Esta classificação deve seguir
conhecimentos de mineração de dados e experiências passadas, tanto do autor como de
grupos externos, com o problema em questão.

Esses "modelos compatíveis" querem dizer modelos que parecerem produtivos para atingir os objetivos de negócio (notavelmente de regressão, para a tarefa que temos em mãos). Lembrando o objetivo de negócio:

> Produzir um preditor robusto para o preço de ações na bolsa de
Nova York.

#### Quais modelos de regressao devem ser considerados?

- Linear regression
- Logistic regression
- Neural networks
- Decision tree / random forests

#### Sobre cada modelo, quais são qualidade procuradas?

- Automated (no need to search for heuristics for augmentation) --- neural networks shine here
- Adaptation to the NYSE data --- no idea
- Interpretability (decision trees may be best here)
- Computational speed, Memory usage(dataset is pretty big) --- linear regression may be best here

#### Quais modelos podem ser desconsiderados

- Logistic regression (até onde eu sei é usado mais para classificação)
- Neural networks (é mais complexa que o necessário)

#### What about regularization?

- Have to decide on the parameters (for example for ridge regression, the norm of the weights is used)

#### Also, there needs to be thought on data normalization

- Gaussian normalization.
- Attribute weights to each feature. I think they should be different. 

#### Passo 3

Criar um ambiente de testes para os modelos escolhidos.

Entre regressão linear e árvores de decisão, decidir ir com regressão linear mesmo. Tem mais chances de se dar bem com o modelo. Penso que a kernelização não será um problema (posso utilizar kernel polinomial misto até grau 3).

A tarefa se torna então criar um ambiente (código Python) para a testagem do modelo com z-normalização.

Note que não é necessário aplicar o modelo aqui.

In [11]:
# Todo o processo mencionado acima é feito nessa função
filtPricesDf = fetchFiltPricesDf(pricesDf)

/usr/lib/python3/dist-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Uma coisa que apareceu aqui é que não parece razoável simplesmente criar um modelo de regressão sobre os dados normalizados dessa forma. Há uma cauda muito pesada para a direita nas curvas gaussianas.

Estou pensando em resolver isso através de uma etapa prévia de agrupamento na etapa de treinamento e também na etapa de testes. Isto é, primeiro se distingue a qual grupo o ponto pertence, e depois se passa o resultado para o modelo de regressão em si.

Penso que não será muito complicado fazer isso.

Em primeiro lugar, porém, vale a pena testar o modelo como está.

Have to divide the dataset into three parts: training, validation and testing.

In [12]:
X_train, X_test, y_train, y_test = splitTrainTest(filtPricesDf)

That is all for this notebook. The rest will be put in the next phase.